In [1]:
%env CUDA_VISIBLE_DEVICES = 3

env: CUDA_VISIBLE_DEVICES=3


In [10]:
%load_ext autoreload
%autoreload 2
import sys; 
sys.path.extend(['/home/meet/FlowMatchingTests/conditional-flow-matching/'])

import matplotlib.pyplot as plt
import numpy as np
import torch
from torchcfm.conditional_flow_matching import *
from physics_flow_matching.unet import UNetModel
from torchdiffeq import odeint
from einops import rearrange

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
for v in ["u, v, w"]:
    

In [3]:
a, b, c = np.load("/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_u_y70_41-220_out.npy"), np.load("/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_v_y70_41-220_out.npy"), np.load("/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_w_y70_41-220_out.npy")
label = np.concatenate((a,b,c), axis=1)
m, std = label.mean(axis=(0,2,3), keepdims=True), label.std(axis=(0,2,3), keepdims=True)

In [4]:
a, b, c = np.load("/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_u_y10-40-70_221-259_out.npy")[:, 1:], np.load("/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_v_y10-40-70_221-259_out.npy")[:, 1:], np.load("/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_w_y10-40-70_221-259_out.npy")[:, 1:]
test_label = np.concatenate((a,b,c), axis=1)

In [5]:
a, b, c = np.load("/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_u_y40_41-220_out.npy"), np.load("/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_v_y40_41-220_out.npy"), np.load("/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_w_y40_41-220_out.npy")
data = np.concatenate((a,b,c), axis=1)
m_, std_ = data.mean(axis=(0,2,3), keepdims=True), data.std(axis=(0,2,3), keepdims=True)

In [6]:
a, b, c = np.load("/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_u_y10-40-70_221-259_out.npy")[:, :1], np.load("/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_v_y10-40-70_221-259_out.npy")[:, :1], np.load("/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_w_y10-40-70_221-259_out.npy")[:, :1]
test_data = np.concatenate((a,b,c), axis=1)
X = (test_data - m_)/std_

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
exp=1
iteration = 15
ot_cfm_model = UNetModel(dim=[3, 160, 200],
                         channel_mult=[1,2,4,4],
                         num_channels=128,
                         num_res_blocks=2,
                         num_head_channels=64,
                         attention_resolutions="40, 20, 10",
                         dropout=0.2,
                         use_new_attention_order=True,
                         use_scale_shift_norm=True)
state = torch.load(f"/home/meet/FlowMatchingTests/conditional-flow-matching/physics_flow_matching/vf_vf/exps/exp_{exp}/saved_state/checkpoint_{iteration}.pth")
ot_cfm_model.load_state_dict(state["model_state_dict"])
ot_cfm_model.to(device)
ot_cfm_model.eval();

/tmp/ipykernel_2550996/1107988862.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(f"/home/meet/FlowMatchingTests/conditional-flow-matching/physics_fl

In [19]:
batch_size=20

In [20]:
samples_list = []
for batch in range(X.shape[0]//batch_size):
    with torch.no_grad():
        out = ot_cfm_model(torch.tensor(torch.tensor(X[batch*batch_size:(batch+1)*batch_size], device=device))).detach().cpu().numpy()
    out *= std
    out += m
    samples_list.append(out)

/tmp/ipykernel_2550996/3540767277.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  out = ot_cfm_model(torch.tensor(torch.tensor(X[batch*batch_size:(batch+1)*batch_size], device=device))).detach().cpu().numpy()


In [21]:
samples = np.concatenate(samples_list)

In [22]:
np.save(f"/home/meet/FlowMatchingTests/conditional-flow-matching/physics_flow_matching/vf_vf/exps/exp_{exp}/samples_{iteration}epch", samples[:, 0])

In [171]:
# for i in range(0,100,5):
#     plt.figure()
#     plt.imshow(out[i, 0, 0])

In [172]:
# np.save(f"/home/meet/FlowMatchingTests/conditional-flow-matching/physics_flow_matching/wp_ws/exps/FM_exps/exp_{exp}/samples_{iteration}iter_7000", samples[:, 0])